# Conversation between DialogueGpt and Leolani

In this notebook, we create a loop in which DialogGpt has a conversation with Leolani. We send Leolani's initial prompt to DialogueGpt to start a conversation.
Next we capture Leolani's response from the brain and DialogueGpt's response constinuously until we meet the stop condition. Extracted triples are posted to a brain called DialueGpt.

In principle, this conversation can go on forever. At the end, we save the scenario in EMISSOR.

Before running, start GraphDB and make sure that there is a sandbox repository.
GraphDB can be downloaded from:

https://graphdb.ontotext.com

## Import the necessary modules

In [1]:
import json
import os
import time
import uuid
from datetime import date
from datetime import datetime
from random import getrandbits, choice
import pathlib
import pprint

# general imports for EMISSOR and the BRAIN
import emissor as em
import requests
from cltl import brain
from cltl.brain.long_term_memory import LongTermMemory
from cltl.brain.utils.helper_functions import brain_response_to_json
from cltl.combot.backend.api.discrete import UtteranceType
from cltl.reply_generation.data.sentences import GREETING, ASK_NAME, ELOQUENCE, TALK_TO_ME
from cltl.reply_generation.lenka_replier import LenkaReplier
from cltl.triple_extraction.api import Chat, UtteranceHypothesis
from emissor.persistence import ScenarioStorage
from emissor.representation.annotation import AnnotationType, Token, NER
from emissor.representation.container import Index
from emissor.representation.scenario import Modality, ImageSignal, TextSignal, Mention, Annotation, Scenario

/Users/piek/PycharmProjects/cltl-chatbots/venv/lib/python3.7/site-packages/rdflib_jsonld/__init__.py:12: DeprecationWarning: The rdflib-jsonld package has been integrated into rdflib as of rdflib==6.0.1.  Please remove rdflib-jsonld from your project's dependencies.
  DeprecationWarning,
[nltk_data] Downloading package punkt to /Users/piek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Import the chatbot utility functions

In [2]:
import sys
import os

src_path = os.path.abspath(os.path.join('..'))
if src_path not in sys.path:
    sys.path.append(src_path)

#### The next utils are needed for the interaction and creating triples and capsules
import chatbots.util.driver_util as d_util
import chatbots.util.capsule_util as c_util
import chatbots.intentions.talk as talk
import chatbots.intentions.get_to_know_you as friend

## Import a conversation agent pipeline

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-medium')
model = AutoModelForCausalLM.from_pretrained('microsoft/DialoGPT-medium')

#### Needed to suppress messages from DialgGPT
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

## Standard initialisation of a scenario

We initialise a scenario in the standard way by creating a unique folder and setting the AGENT and HUMAN_NAME and HUMAN_ID variables. Throughout this scenario, the HUMAN_NAME and HUMAN_ID will be used as the source for the utterances.

In [5]:
from random import getrandbits
import requests
##### Setting the location
place_id = getrandbits(8)
location = None
try:
    location = requests.get("https://ipinfo.io").json()
except:
    print("failed to get the IP location")
    
##### Setting the agents
AGENT = "Leolani2"
HUMAN_NAME = "DialogGPT"
HUMAN_ID = "DialogGPT"

### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H:%M:%S")

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
# Find the repository root dir
parent, dir_name = (d_util.__file__, "_")
while dir_name and dir_name != "src":
    parent, dir_name = os.path.split(parent)
root_dir = parent
scenario_path = os.path.abspath(os.path.join(root_dir, 'data'))

if not os.path.exists(scenario_path) :
    os.mkdir(scenario_path)
    print("Created a data folder for storing the scenarios", scenario_path)

### Define the folders where the images and rdf triples are saved
imagefolder = scenario_path + "/" + scenario_id + "/" + "image"
rdffolder = scenario_path + "/" + scenario_id + "/" + "rdf"

### Create the scenario folder, the json files and a scenarioStorage and scenario in memory
scenarioStorage = d_util.create_scenario(scenario_path, scenario_id)
scenario_ctrl = scenarioStorage.create_scenario(scenario_id, int(time.time() * 1e3), None, AGENT)

Directories for 2021-11-30-09:08:06 created in /Users/piek/PycharmProjects/cltl-chatbots/data


## Specifying the BRAIN

We specify the BRAIN in GraphDB and use the scenario path just defined for storing the RDF triple produced in EMISSOR.

If you set *clear_all* to *True*, the sandbox triple store is emptied (memory erased) and the basic ontological models are reloaded. Setting it to *False* means you add things to the current memory.

In [6]:
log_path = pathlib.Path(rdffolder)
my_brain = brain.LongTermMemory(address="http://localhost:7200/repositories/dialogGpt",
                                log_dir=log_path,
                                clear_all=True)

2021-11-30 09:08:08,437 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Booted
2021-11-30 09:08:10,960 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Uploading ontology to brain
2021-11-30 09:08:13,238 -     INFO -  cltl.brain.basic_brain.ThoughtGenerator - Booted
2021-11-30 09:08:13,241 -     INFO -  cltl.brain.basic_brain.LocationReasoner - Booted
2021-11-30 09:08:13,243 -     INFO -      cltl.brain.basic_brain.TypeReasoner - Booted
2021-11-30 09:08:13,245 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Booted
2021-11-30 09:08:13,583 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Computed trust for all known agents


## Create an instance of a replier

In [7]:
replier = LenkaReplier()

2021-11-30 09:08:14,803 -     INFO -   cltl.reply_generation.api.LenkaReplier - Booted


## Initialise a chat with the HUMAN_ID to keep track of the dialogue history

In [8]:
chat = Chat(HUMAN_ID)

2021-11-30 09:08:15,635 -     INFO - cltl.triple_extraction.api.Chat (DialogGPT)         000 - << Start of Chat with DialogGPT >>


## Start the interaction

In [9]:
print_details=False

t1 = datetime.now()

#### Initial prompt by the system from which we create a TextSignal and store it
leolani_prompt = f"{choice(TALK_TO_ME)}"
print('\n\t'+AGENT + ": " + leolani_prompt)
textSignal = d_util.create_text_signal(scenario_ctrl, leolani_prompt)
scenario_ctrl.append_signal(textSignal)

# encode the new Leolani input, add the eos_token and return a tensor in Pytorch
bot_input_ids = tokenizer.encode(leolani_prompt + tokenizer.eos_token, return_tensors='pt')
# append the new user input tokens to the chat history
#bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1) if step > 0 else new_input_ids
# generated a response while limiting the total chat history to 1000 tokens, 
chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

utterance = ""
#### Get input and loop
while (datetime.now()-t1).seconds <= 120:
    ###### Getting the next input signals
    utterance = format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True))
    if not utterance:
        leolani_prompt = f"{choice(TALK_TO_ME)}"
        print('\n\t'+AGENT + ": " + leolani_prompt)
        textSignal = d_util.create_text_signal(scenario_ctrl, leolani_prompt)
        scenario_ctrl.append_signal(textSignal)
        # encode the new Leolani input, add the eos_token and return a tensor in Pytorch
        new_input_ids = tokenizer.encode(leolani_prompt + tokenizer.eos_token, return_tensors='pt')
        # append the new user input tokens to the chat history
        bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1)
        # generated a response while limiting the total chat history to 1000 tokens, 
        chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    else:
        print('\n\t'+HUMAN_NAME + ": " + utterance)
        textSignal = d_util.create_text_signal(scenario_ctrl, utterance)
        scenario_ctrl.append_signal(textSignal)

        #### Process input and generate reply

        capsule, leolani_prompt = talk.process_text_and_reply(scenario_ctrl,
                               place_id,
                               location,
                               HUMAN_ID,
                               textSignal,
                               chat,
                               replier,
                               my_brain,
                               print_details)

        print('\n\t'+AGENT + ": " + leolani_prompt)
        textSignal = d_util.create_text_signal(scenario_ctrl, leolani_prompt)
        scenario_ctrl.append_signal(textSignal)
        
        # encode the new Leolani input, add the eos_token and return a tensor in Pytorch
        new_input_ids = tokenizer.encode(leolani_prompt + tokenizer.eos_token, return_tensors='pt')
        # append the new user input tokens to the chat history
        bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1)
        # generated a response while limiting the total chat history to 1000 tokens, 
        chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)



	Leolani2: Let's chat!
I'm on mobile, but I'll PM you.

	DialogGPT: I'm on mobile, but I'll PM you.
2021-11-30 09:08:17,946 -     INFO -               cltl.triple_extraction.api - Started POS tagger
2021-11-30 09:08:17,967 -     INFO -               cltl.triple_extraction.api - Started NER tagger
2021-11-30 09:08:17,970 -     INFO -               cltl.triple_extraction.api - Loaded grammar
2021-11-30 09:08:19,375 -     INFO - cltl.triple_extraction.api.Chat (DialogGPT)         001 -  DialogGPT: "I'm on mobile, but I'll PM you."


Couldn't parse input



	Leolani2: Interesting
I'm on mobile, but I'll PM you.

	DialogGPT: I'm on mobile, but I'll PM you.
2021-11-30 09:08:21,831 -     INFO - cltl.triple_extraction.api.Chat (DialogGPT)         002 -  DialogGPT: "I'm on mobile, but I'll PM you."


Couldn't parse input



	Leolani2: Good to know
I'm on mobile, but I'll PM you.

	DialogGPT: I'm on mobile, but I'll PM you.
2021-11-30 09:08:24,478 -     INFO - cltl.triple_extraction.api.Chat (DialogGPT)         003 -  DialogGPT: "I'm on mobile, but I'll PM you."


...

Couldn't parse input



	Leolani2: But at what cost?
I see

	DialogGPT: I see
2021-11-30 09:08:38,502 -     INFO - cltl.triple_extraction.api.Chat (DialogGPT)         009 -  DialogGPT: "I see"


Couldn't parse input



	Leolani2: and then the wolves came


	Leolani2: Tell me anything, I love learning things


NameError: name 'conv' is not defined

In [14]:
conv.generated_responses

["I'm a student at the University of Texas at Austin.",
 "I'm a student at the University of Texas at Austin.",
 "I don't know. I'm not a student at the University of Texas at Austin.",
 "I'm a student at the University of Texas at Austin.",
 '',
 'I love you',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

## Save the scenario data

In [15]:
scenario_ctrl.scenario.ruler.end = int(time.time() * 1e3)
scenarioStorage.save_scenario(scenario_ctrl)

## End of notebook